# Duterte SONAs

This processes all collated Duterte SONA. Reminder to run the [Philippines SONA](https://github.com/pmagtulis/ph-sona.git) scraper file to collect the **merged** CSV file here.

## Do all your imports

In [1]:
import pandas as pd
import numpy as np
import re
import altair as alt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import stopwordsiso as stopwords

## Read CSV

In [3]:
merged= pd.read_csv('merged.csv')
merged

,Unnamed: 0,president,date,title,link,venue,session,speech
0,0,Manuel L. Quezon,"November 25, 1935",Message to the First Assembly on National Defense,http://www.officialgazette.gov.ph/1935/11/25/m...,"Legislative Building, Manila","First National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
1,1,Manuel L. Quezon,"June 16, 1936",On the Country’s Conditions and Problems,http://www.officialgazette.gov.ph/1936/06/16/m...,"Legislative Building, Manila","First National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
2,2,Manuel L. Quezon,"October 18, 1937","Improvement of Philippine Conditions, Philippi...",http://www.officialgazette.gov.ph/1937/10/18/m...,"Legislative Building, Manila","First National Assembly, Second Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
3,3,Manuel L. Quezon,"January 24, 1938",Revision of the System of Taxation,http://www.officialgazette.gov.ph/1938/01/24/m...,"Legislative Building, Manila","First National Assembly, Third Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
4,4,Manuel L. Quezon,"January 24, 1939",The State of the Nation and Important Economic...,http://www.officialgazette.gov.ph/1939/01/24/m...,"Legislative Building, Manila","Second National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
...,...,...,...,...,...,...,...,...
79,79,Rodrigo Roa Duterte,"July 23, 2018",Third State of the Nation Address,https://www.officialgazette.gov.ph/2018/07/23/...,"Batasang Pambansa, Quezon City","Seventeenth Congress, Third Session",\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
80,80,Rodrigo Roa Duterte,"July 22, 2019",Fourth State of the Nation Address,https://www.officialgazette.gov.ph/2019/07/22/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, First Session",\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
81,81,Rodrigo Roa Duterte,"July 27, 2020",Fifth State of the Nation Address,https://www.officialgazette.gov.ph/2020/07/27/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, Second Session",\n\n\n\n\n\n\n5TH STATE OF THE NATION ADDRESS ...
82,82,Rodrigo Roa Duterte,"July 26, 2021",Sixth State of the Nation Address,https://www.officialgazette.gov.ph/2021/07/26/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, Third Session",\n\n\tState of the Nation Address of \n\tRodri...


## Isolate 'Duterte' speeches

The merged file contains all speeches by Philippine presidents since 1935. 

In [4]:
duterte = merged[(merged['president'] == 'Rodrigo Roa Duterte')] #Duterte

## Text analysis

Now, we can proceed with the text analysis proper. First stop, we set the parameters in the immediate cell below, most importantly the stopwords we want our analysis to disregard.

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    return text #removes all numbers

In [173]:
y_columns = ['president', 'speeches']
BINARY=False
NGRAM_RANGE=(1,1)
MIN_DF=0
STPWORDS=stopwords.stopwords(["en", "tl"]) #removes Tagalog stopwords
STPWORDS.update(['yung', 'iyan', 'yan', 'diyan', 'applause', 'laughter', 'palakpakan', 'rin', 'din', 'po',
                'pong', 'pang', 'pa', 'nang', 'ng', 'pag',
                'kapag', 'nga', 'naman', 'natin', 'kayo',
                'nating', 'natin', 'tayong', 'lang',
                'ninyo', 'itong']) #adds more Tagalog stopwords not included in the package 

vectorizer = CountVectorizer(
    stop_words=STPWORDS,
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)

## Vectorizing

Simple counting of words that occur in a speech.

In [174]:
X = vectorizer.fit_transform(duterte['speech'])
X

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


<6x6495 sparse matrix of type '<class 'numpy.int64'>'
	with 11603 stored elements in Compressed Sparse Row format>

In [175]:
duterte_vectors = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# [print(x) for x in marcosjr.speech]
duterte_vectors.round(2)

,aabot,aakyat,aalis,aambush,aano,aaway,abandon,abdul,abiding,ability,...,youth,yun,zamboanga,zamora,zeal,zeroing,zhao,zone,zones,zoom
0,0,0,0,0,1,1,0,0,0,1,...,0,3,0,0,0,0,0,1,1,0
1,0,0,0,1,0,2,0,0,0,1,...,3,5,2,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,2,0,0
4,0,0,1,0,0,0,0,0,1,2,...,0,0,0,0,1,1,0,0,1,0
5,0,1,1,0,0,0,1,0,1,2,...,0,3,2,1,0,0,0,1,1,2


In [176]:
duterte_vectors = duterte_vectors.transpose() #swapping columns and row positions

In [177]:
duterte_vectors.columns = ['SONA1', 'SONA2', 'SONA3', 'SONA4', 'SONA5', 'SONA6']
duterte_vectors.sort_values('SONA1', ascending=False).head(20)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6
government,32,47,21,29,42,35
kasi,30,21,0,14,9,14
wala,30,24,0,22,11,19
time,22,33,8,26,22,36
people,21,31,15,17,25,29
country,21,22,13,6,28,43
president,15,10,4,14,9,18
talaga,14,13,1,11,6,21
drug,14,1,5,3,1,10
alam,14,16,1,13,17,16


## Add a 'total' mention column

Totally optional, just in case you wanted to find the total number of mentions.

In [178]:
duterte_vectors['total'] = duterte_vectors.SONA1 + duterte_vectors.SONA2 + duterte_vectors.SONA3 + duterte_vectors.SONA4 + duterte_vectors.SONA5 + duterte_vectors.SONA6

In [179]:
duterte_vectors = duterte_vectors.sort_values('total', ascending=False)
duterte_vectors.head(15)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6,total
government,32,47,21,29,42,35,206
time,22,33,8,26,22,36,147
people,21,31,15,17,25,29,138
country,21,22,13,6,28,43,133
wala,30,24,0,22,11,19,106
congress,7,13,12,23,23,26,104
law,8,21,11,15,18,16,89
kasi,30,21,0,14,9,14,88
filipino,5,11,12,8,15,27,78
alam,14,16,1,13,17,16,77


# TF-IDF

## Duterte speeches

In [180]:
vectorizer = TfidfVectorizer(
    stop_words=STPWORDS, 
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)
X = vectorizer.fit_transform(duterte['speech'])
duterte_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#[print(x) for x in speeches.sentence]
duterte_idf.round(2)

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


,aabot,aakyat,aalis,aambush,aano,aaway,abandon,abdul,abiding,ability,...,youth,yun,zamboanga,zamora,zeal,zeroing,zhao,zone,zones,zoom
0,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01,...,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00
1,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.00,0.01,...,0.02,0.04,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,...,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00
4,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.02,...,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.01,0.00
5,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.01,...,0.00,0.02,0.02,0.01,0.00,0.00,0.00,0.01,0.01,0.02


In [181]:
duterte_idf2 = duterte_idf.transpose()
duterte_idf2.columns = ['SONA1', 'SONA2', 'SONA3', 'SONA4', 'SONA5', 'SONA6']

In [182]:
duterte_idf2.sort_values('SONA4', ascending=False).head(15)

,SONA1,SONA2,SONA3,SONA4,SONA5,SONA6
government,0.196331,0.246705,0.220582,0.191741,0.263856,0.163957
time,0.134977,0.173218,0.084031,0.171906,0.138210,0.168642
wala,0.212433,0.145396,0.000000,0.167881,0.079758,0.102726
tsk,0.000000,0.000000,0.000000,0.158781,0.000000,0.017307
congress,0.042947,0.068238,0.126047,0.152071,0.144493,0.121797
money,0.006135,0.068238,0.042016,0.132235,0.018847,0.070267
people,0.128842,0.162720,0.157559,0.112400,0.157057,0.135850
davao,0.077892,0.084815,0.000000,0.106834,0.014501,0.113539
kasi,0.212433,0.127222,0.000000,0.106834,0.065256,0.075693
national,0.055218,0.062989,0.105039,0.105788,0.087952,0.051529


## Looking for specific words

In this part, we are looking for specific words that we think made a mark during Duterte SONAs, whether because they are often mentioned, or because it is unusual for the Chief Executive to say it. 

We also include here words that we think were said because they were the topic at hand at the time the speech was delivered.

In [183]:
duterte_slice = duterte_idf[['drug', 'drugs', 'mining', 'pandemic', 'covid', 'rice']]
duterte_slice.sort_index().round(decimals=2)

,drug,drugs,mining,pandemic,covid,rice
0,0.09,0.05,0.03,0.00,0.00,0.02
1,0.01,0.04,0.16,0.00,0.00,0.00
2,0.05,0.05,0.07,0.00,0.00,0.14
3,0.02,0.03,0.00,0.00,0.00,0.02
4,0.01,0.04,0.00,0.21,0.15,0.00
5,0.05,0.09,0.00,0.15,0.17,0.01


In [184]:
duterte_slice = duterte_slice.stack().reset_index()
duterte_slice = duterte_slice.rename(columns={'level_0': 'sona_no','level_1': 'term', 'tfidf': 'term', 0: 'tfidf'})
duterte_slice.head()

,sona_no,term,tfidf
0,0,drug,0.085895
1,0,drugs,0.049083
2,0,mining,0.028706
3,0,pandemic,0.000000
4,0,covid,0.000000


In [185]:
top_tfidf = duterte_slice.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
top_tfidf.head()

,sona_no,term,tfidf
0,0,drug,0.085895
1,0,drugs,0.049083
2,0,mining,0.028706
5,0,rice,0.016399
3,0,pandemic,0.000000


## Chart it

In [186]:
# # Terms in this list will get a red dot in the visualization
term_list = ['drug', 'drugs']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600, height=400)

alt.LayerChart(...)

## Entire SONAs

In here, we do the same thing for all of SONA *without* isolating key words.

In [187]:
duterte_idf = duterte_idf.stack().reset_index()
duterte_idf

,level_0,level_1,0
0,0,aabot,0.000000
1,0,aakyat,0.000000
2,0,aalis,0.000000
3,0,aambush,0.000000
4,0,aano,0.013821
...,...,...,...
38965,5,zeroing,0.000000
38966,5,zhao,0.000000
38967,5,zone,0.007306
38968,5,zones,0.007306


In [188]:
duterte_idf = duterte_idf.rename(columns={'level_0': 'sona_no','level_1': 'term', 0: 'tfidf'})
duterte_idf

,sona_no,term,tfidf
0,0,aabot,0.000000
1,0,aakyat,0.000000
2,0,aalis,0.000000
3,0,aambush,0.000000
4,0,aano,0.013821
...,...,...,...
38965,5,zeroing,0.000000
38966,5,zhao,0.000000
38967,5,zone,0.007306
38968,5,zones,0.007306


In [189]:
all_dutertesona = duterte_idf.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
all_dutertesona.head()

,sona_no,term,tfidf
3025,0,kasi,0.212433
6362,0,wala,0.212433
2343,0,government,0.196331
5972,0,time,0.134977
1205,0,country,0.128842


In [190]:
# # Terms in this list will get a red dot in the visualization
term_list = ['drug', 'drugs']

# adding a little randomness to break ties in term ranking
all_dutertesona_plusRand = all_dutertesona.copy()
all_dutertesona_plusRand['tfidf'] = all_dutertesona_plusRand['tfidf'] + np.random.rand(all_dutertesona.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(all_dutertesona_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
duterte_chart = (heatmap + circle + text).properties(width = 500, height=300)
duterte_chart

alt.LayerChart(...)

## Aquino

In [191]:
aquino = merged[(merged['president'] == 'Benigno S. Aquino III')] #Aquino

In [ ]:
vectorizer = TfidfVectorizer(
    stop_words=STPWORDS, 
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)
X = vectorizer.fit_transform(aquino['speech'])
aquino_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#[print(x) for x in speeches.sentence]
aquino_idf.round(2)

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


In [ ]:
aquino_idf = aquino_idf.stack().reset_index()
aquino_idf = aquino_idf.rename(columns={'level_0': 'sona_no','level_1': 'term', 0: 'tfidf'})
aquino_idf = aquino_idf.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
aquino_idf.head()

In [ ]:
# adding a little randomness to break ties in term ranking
aquino_idf_plusRand = aquino_idf.copy()
aquino_idf_plusRand['tfidf'] = aquino_idf_plusRand['tfidf'] + np.random.rand(aquino_idf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(aquino_idf_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
aquino_chart = (heatmap + text).properties(width = 500, height=300)
aquino_chart

## Combine the two charts

In [ ]:
alt.vconcat(aquino_chart, duterte_chart)

In [ ]:
aquino_chart | duterte_chart